# Rigid Body Simulator

Credit to Professor Bo Zhu, on whose COSC 89 Course Notes the following code is based on

## Geometry in space

In [72]:
import taichi as ti
import numpy as np

ti.init()

# variables and fields to keep track of
num_particles = 1
dt = 1e-3
substeps = 10
mass = 1

# Inertial tensor
I = 10
# position - center of mass
x = ti.Vector.field(n=2, dtype=ti.f32, shape=num_particles)
# x = ti.Vector([0.5, 0.5])
# vertices - relative to center of mass -- will have to change if more than one solid
vertices_0 = ti.Vector.field(n=2, dtype=ti.f32, shape=num_particles*3)
rotated_vertices = ti.Vector.field(n=2, dtype=ti.f32, shape=num_particles*3)
# force
force = ti.Vector.field(n=3, dtype=ti.f32, shape=num_particles)
# torque
torque = ti.Vector.field(n=3, dtype=ti.f32, shape=num_particles)
# velocity
v = ti.Vector.field(n=3, dtype=ti.f32, shape=num_particles)
# orientation
theta = ti.field(dtype=ti.f32, shape=num_particles)
# angular velocity
w = ti.Vector.field(n=3, dtype=ti.f32, shape=num_particles)

@ti.kernel
def init_pos():
    x[0] = ti.Vector([0.5, 0.5])
    # equilateral triangle
    vertices_0[0] = ti.Vector([0, 0.1])
    vertices_0[1] = ti.Vector([-0.0866, -0.05])
    vertices_0[2] = ti.Vector([0.0866, -0.05])
    rotated_vertices[0] = ti.Vector([0, 0.1])
    rotated_vertices[1] = ti.Vector([-0.0866, -0.05])
    rotated_vertices[2] = ti.Vector([0.0866, -0.05])
    # initialize velocities
    v[0] = ti.Vector([0, 0, 0])
    # initialize force
    force[0] = ti.Vector([0,0,0])
    

@ti.func
def calculate_force():
    # gravity
    force = force - ti.Vector([0, -2, 0])

@ti.func
def rotate():
    pass

@ti.func
def calculate_torque():
    pass

@ti.kernel
def substep():
    calculate_force()
    calculate_torque()
    # update velocity
    v[0] = v[0] + dt * force[0] / mass
    # update angular velocity
    w[0] = w[0] + dt * torque[0] / I
    # update position
    x[0] = x[0] + dt * v[0]
    # update orientation
    theta[0] = theta[0] * dt * w[0]
    # update vertices
    rotate()

def main():
    gui = ti.GUI('Rigid Body', (600, 600))
    # initialize vertices
    init_pos()
    
    while gui.running:
        for e in gui.get_events(ti.GUI.PRESS):
            pass
            
        # Update for a time step
        for step in range(substeps):
            substep()

        gui.triangle(a=x[0]+rotated_vertices[0], b=x[0]+rotated_vertices[1], c=x[0]+rotated_vertices[2], color=0xED553B)
        
        gui.show()
    
if __name__ == '__main__':
    main()

[Taichi] Starting on arch=x64


Code from stackoverflow on finding the inertial tensor using triangles

https://stackoverflow.com/questions/41592034/computing-tensor-of-inertia-in-2d

## Particle Sand

In [164]:
import taichi as ti

ti.init()

max_particles = 100
num_particles = ti.field(dtype=ti.i32, shape=())
dt = 1e-3
substeps = 10

ks = 2e2
kd = 0.5e1

x = ti.Vector.field(2, dtype=ti.f32, shape=max_particles)
v = ti.Vector.field(2, dtype=ti.f32, shape=max_particles)
f = ti.Vector.field(2, dtype=ti.f32, shape=max_particles)
r = ti.field(dtype=ti.f32, shape=max_particles)
m = ti.field(dtype=ti.f32, shape=max_particles)


@ti.kernel
def substep():
    # find force
    for i in range(num_particles[None]):
        f[i] = [0,-2] # reset force, add in gravity
        # boundary check
        phi = x[i][1] - r[i]/640
        # phi = (ti.Vector([x[i][0], 0]) - x[i]).norm() - r[i]/640
        if phi < 0: # lower boundary
            f[i] += ks * (phi) * ti.Vector([0,-1])
            f[i] += kd * (ti.Vector([0,0]) - v[i]).dot(ti.Vector([0,-1])) * ti.Vector([0, -1])
            ###
            # How do I make it so the resting state isn't slightly submerged
            ###

        for j in range(i+1, num_particles[None]):
            # check for collision
            phi = (x[i] - x[j]).norm() - r[i]/640 - r[j]/640
#             print("j = ", j, "i = ", "pos[i] = ", x[i], "pos[j] = ", x[j], "phi = ", phi)
            if phi < 0:
                force = ks * (phi) * (x[j] - x[i]).normalized()
#                 force_spring = ks * (phi) * (ti.Vector([0, -1]))
                f[i] += force_spring
                f[j] -= force_spring
#                 force_damp = kd * (ti.Vector([0,0]) - v[i]).dot(ti.Vector([0,-1])) * ti.Vector([0, -1])
                normal = (x[j] - x[i]).normalized()
#                 normal = ti.Vector([0, -1])
                force_damp = kd * (v[j] - v[i]).dot(normal) * normal
                f[i] += force_damp
                f[j] -= force_damp
                
    for i in range(num_particles[None]):
        # update v
        v[i] = v[i] + dt * f[i] / m[i]
        # update x
        x[i] = x[i] + dt * v[i]

        
@ti.kernel
def new_particle(pos_x: ti.f32, pos_y: ti.f32):
    particle_id = num_particles[None]
    if particle_id < max_particles:
        x[particle_id] = [pos_x, pos_y]
        v[particle_id] = [0, 0]
        m[particle_id] = 1
        r[particle_id] = 10
        num_particles[None] += 1
        

def main():
    gui = ti.GUI('Particle Sand', (640, 640))
    
    new_particle(0.3, 0.5)
    
    while gui.running:
        for e in gui.get_events(ti.GUI.PRESS):
            if e.key == ti.GUI.UP:
                ks *= 1.1
            elif e.key == ti.GUI.DOWN:
                ks /= 1.1
            elif e.key == ti.GUI.LMB:
                new_particle(e.pos[0], e.pos[1])
            
        # Move stuff
        for step in range(substeps):
            substep()
             
        # Draw particles
        X = x.to_numpy()
        for i in range(num_particles[None]):
            gui.circle(pos=X[i], radius = r[i])

        # show the current values
        gui.text(
            content='Click anywhere on screen to add a new mass',
            pos=(0,0.99),
            color=0xffffff)
        
        gui.show()
    
if __name__ == '__main__':
    main()

[Taichi] Starting on arch=x64
 [0.300000, 0.027465] pos[j] =  [0.300000, 0.027465] phi =  -0.031250
j =  0 i =  pos[i] =  [0.300000, 0.028133] pos[j] =  [0.300000, 0.028133] phi =  -0.031250
j =  0 i =  pos[i] =  [0.300000, 0.028800] pos[j] =  [0.300000, 0.028800] phi =  -0.031250
j =  0 i =  pos[i] =  [0.300000, 0.029464] pos[j] =  [0.300000, 0.029464] phi =  -0.031250
j =  0 i =  pos[i] =  [0.300000, 0.030126] pos[j] =  [0.300000, 0.030126] phi =  -0.031250
j =  0 i =  pos[i] =  [0.300000, 0.030787] pos[j] =  [0.300000, 0.030787] phi =  -0.031250
j =  0 i =  pos[i] =  [0.300000, 0.031445] pos[j] =  [0.300000, 0.031445] phi =  -0.031250
j =  0 i =  pos[i] =  [0.300000, 0.032101] pos[j] =  [0.300000, 0.032101] phi =  -0.031250
j =  0 i =  pos[i] =  [0.300000, 0.032756] pos[j] =  [0.300000, 0.032756] phi =  -0.031250
j =  0 i =  pos[i] =  [0.300000, 0.033408] pos[j] =  [0.300000, 0.033408] phi =  -0.031250
j =  0 i =  pos[i] =  [0.300000, 0.034059] pos[j] =  [0.300000, 0.034059] phi =  

j =  1 i =  pos[i] =  [0.300000, -0.000880] pos[j] =  [0.304687, 0.397567] phi =  0.367225
j =  1 i =  pos[i] =  [0.300000, -0.000479] pos[j] =  [0.304687, 0.397507] phi =  0.366764
j =  1 i =  pos[i] =  [0.300000, -0.000078] pos[j] =  [0.304687, 0.397445] phi =  0.366301
j =  1 i =  pos[i] =  [0.300000, 0.000321] pos[j] =  [0.304687, 0.397381] phi =  0.365838
j =  1 i =  pos[i] =  [0.300000, 0.000720] pos[j] =  [0.304687, 0.397315] phi =  0.365374
j =  1 i =  pos[i] =  [0.300000, 0.001117] pos[j] =  [0.304687, 0.397247] phi =  0.364908
j =  1 i =  pos[i] =  [0.300000, 0.001513] pos[j] =  [0.304687, 0.397177] phi =  0.364442
j =  1 i =  pos[i] =  [0.300000, 0.001909] pos[j] =  [0.304687, 0.397105] phi =  0.363975
j =  1 i =  pos[i] =  [0.300000, 0.002303] pos[j] =  [0.304687, 0.397031] phi =  0.363507
j =  1 i =  pos[i] =  [0.300000, 0.002695] pos[j] =  [0.304687, 0.396955] phi =  0.363038
j =  1 i =  pos[i] =  [0.300000, 0.003087] pos[j] =  [0.304687, 0.396877] phi =  0.362569
j =  1 

j =  1 i =  pos[i] =  [0.300000, 0.037853] pos[j] =  [0.304687, 0.353705] phi =  0.284637
j =  1 i =  pos[i] =  [0.300000, 0.037879] pos[j] =  [0.304687, 0.353281] phi =  0.284187
j =  1 i =  pos[i] =  [0.300000, 0.037903] pos[j] =  [0.304687, 0.352855] phi =  0.283737
j =  1 i =  pos[i] =  [0.300000, 0.037925] pos[j] =  [0.304687, 0.352427] phi =  0.283288
j =  1 i =  pos[i] =  [0.300000, 0.037945] pos[j] =  [0.304687, 0.351997] phi =  0.282838
j =  1 i =  pos[i] =  [0.300000, 0.037962] pos[j] =  [0.304687, 0.351565] phi =  0.282388
j =  1 i =  pos[i] =  [0.300000, 0.037978] pos[j] =  [0.304687, 0.351131] phi =  0.281938
j =  1 i =  pos[i] =  [0.300000, 0.037992] pos[j] =  [0.304687, 0.350695] phi =  0.281488
j =  1 i =  pos[i] =  [0.300000, 0.038004] pos[j] =  [0.304687, 0.350257] phi =  0.281039
j =  1 i =  pos[i] =  [0.300000, 0.038014] pos[j] =  [0.304687, 0.349817] phi =  0.280589
j =  1 i =  pos[i] =  [0.300000, 0.038022] pos[j] =  [0.304687, 0.349375] phi =  0.280139
j =  1 i =

j =  1 i =  pos[i] =  [0.300000, 0.009939] pos[j] =  [0.304687, 0.243596] phi =  0.202453
j =  1 i =  pos[i] =  [0.300000, 0.009641] pos[j] =  [0.304687, 0.242808] phi =  0.201963
j =  1 i =  pos[i] =  [0.300000, 0.009344] pos[j] =  [0.304687, 0.242018] phi =  0.201471
j =  1 i =  pos[i] =  [0.300000, 0.009047] pos[j] =  [0.304687, 0.241226] phi =  0.200976
j =  1 i =  pos[i] =  [0.300000, 0.008751] pos[j] =  [0.304687, 0.240432] phi =  0.200478
j =  1 i =  pos[i] =  [0.300000, 0.008456] pos[j] =  [0.304687, 0.239636] phi =  0.199977
j =  1 i =  pos[i] =  [0.300000, 0.008161] pos[j] =  [0.304687, 0.238838] phi =  0.199474
j =  1 i =  pos[i] =  [0.300000, 0.007868] pos[j] =  [0.304687, 0.238038] phi =  0.198967
j =  1 i =  pos[i] =  [0.300000, 0.007576] pos[j] =  [0.304687, 0.237236] phi =  0.198458
j =  1 i =  pos[i] =  [0.300000, 0.007285] pos[j] =  [0.304687, 0.236432] phi =  0.197945
j =  1 i =  pos[i] =  [0.300000, 0.006995] pos[j] =  [0.304687, 0.235626] phi =  0.197429
j =  1 i =

j =  1 i =  pos[i] =  [0.300000, -0.004370] pos[j] =  [0.304687, 0.069536] phi =  0.042805
j =  1 i =  pos[i] =  [0.300000, -0.004220] pos[j] =  [0.304687, 0.068388] phi =  0.041509
j =  1 i =  pos[i] =  [0.300000, -0.004068] pos[j] =  [0.304687, 0.067239] phi =  0.040210
j =  1 i =  pos[i] =  [0.300000, -0.003914] pos[j] =  [0.304687, 0.066087] phi =  0.038908
j =  1 i =  pos[i] =  [0.300000, -0.003760] pos[j] =  [0.304687, 0.064933] phi =  0.037602
j =  1 i =  pos[i] =  [0.300000, -0.003605] pos[j] =  [0.304687, 0.063777] phi =  0.036294
j =  1 i =  pos[i] =  [0.300000, -0.003448] pos[j] =  [0.304687, 0.062619] phi =  0.034983
j =  1 i =  pos[i] =  [0.300000, -0.003291] pos[j] =  [0.304687, 0.061459] phi =  0.033669
j =  1 i =  pos[i] =  [0.300000, -0.003132] pos[j] =  [0.304687, 0.060297] phi =  0.032352
j =  1 i =  pos[i] =  [0.300000, -0.002973] pos[j] =  [0.304687, 0.059133] phi =  0.031032
j =  1 i =  pos[i] =  [0.300000, -0.002812] pos[j] =  [0.304687, 0.057967] phi =  0.029709

j =  1 i =  pos[i] =  [0.300000, 0.005876] pos[j] =  [0.304687, -0.055764] phi =  0.030568
j =  1 i =  pos[i] =  [0.300000, 0.005880] pos[j] =  [0.304687, -0.055437] phi =  0.030246
j =  1 i =  pos[i] =  [0.300000, 0.005885] pos[j] =  [0.304687, -0.055100] phi =  0.029915
j =  1 i =  pos[i] =  [0.300000, 0.005889] pos[j] =  [0.304687, -0.054752] phi =  0.029572
j =  1 i =  pos[i] =  [0.300000, 0.005894] pos[j] =  [0.304687, -0.054394] phi =  0.029220
j =  1 i =  pos[i] =  [0.300000, 0.005898] pos[j] =  [0.304687, -0.054026] phi =  0.028857
j =  1 i =  pos[i] =  [0.300000, 0.005902] pos[j] =  [0.304687, -0.053648] phi =  0.028484
j =  1 i =  pos[i] =  [0.300000, 0.005906] pos[j] =  [0.304687, -0.053260] phi =  0.028101
j =  1 i =  pos[i] =  [0.300000, 0.005910] pos[j] =  [0.304687, -0.052861] phi =  0.027708
j =  1 i =  pos[i] =  [0.300000, 0.005914] pos[j] =  [0.304687, -0.052454] phi =  0.027306
j =  1 i =  pos[i] =  [0.300000, 0.005918] pos[j] =  [0.304687, -0.052036] phi =  0.026893

j =  1 i =  pos[i] =  [0.300000, 0.040149] pos[j] =  [0.304687, 0.051669] phi =  -0.018813
j =  1 i =  pos[i] =  [0.300000, 0.040630] pos[j] =  [0.304687, 0.052150] phi =  -0.018813
j =  1 i =  pos[i] =  [0.300000, 0.041113] pos[j] =  [0.304687, 0.052630] phi =  -0.018815
j =  1 i =  pos[i] =  [0.300000, 0.041597] pos[j] =  [0.304687, 0.053107] phi =  -0.018822
j =  1 i =  pos[i] =  [0.300000, 0.042083] pos[j] =  [0.304687, 0.053582] phi =  -0.018832
j =  1 i =  pos[i] =  [0.300000, 0.042570] pos[j] =  [0.304687, 0.054056] phi =  -0.018845
j =  1 i =  pos[i] =  [0.300000, 0.043060] pos[j] =  [0.304687, 0.054527] phi =  -0.018862
j =  1 i =  pos[i] =  [0.300000, 0.043551] pos[j] =  [0.304687, 0.054996] phi =  -0.018882
j =  1 i =  pos[i] =  [0.300000, 0.044043] pos[j] =  [0.304687, 0.055464] phi =  -0.018905
j =  1 i =  pos[i] =  [0.300000, 0.044538] pos[j] =  [0.304687, 0.055929] phi =  -0.018932
j =  1 i =  pos[i] =  [0.300000, 0.045034] pos[j] =  [0.304687, 0.056392] phi =  -0.018962

[Taichi] Starting on arch=x64
